## gencode.v41.annotation.gff3 の読み込み
gff["attributes"]に様々な情報が格納されている。各情報は";"で区切られているので`split(";")`で情報を抽出できる。

In [1]:
import pandas as pd
import numpy as np
import gffpandas.gffpandas as gffpd

In [16]:
annotation = gffpd.read_gff3("gencode.v41.annotation.gff3")
gff = annotation.df
gff = gff.rename(columns={"seq_id":"chr"}) # 染色体の列名を chr に変更

print(len(gff))

gff.head()

3373604


,chr,source,type,start,end,score,strand,phase,attributes
0,chr1,HAVANA,gene,11869,14409,.,+,.,ID=ENSG00000223972.5;gene_id=ENSG00000223972.5...
1,chr1,HAVANA,transcript,11869,14409,.,+,.,ID=ENST00000456328.2;Parent=ENSG00000223972.5;...
2,chr1,HAVANA,exon,11869,12227,.,+,.,ID=exon:ENST00000456328.2:1;Parent=ENST0000045...
3,chr1,HAVANA,exon,12613,12721,.,+,.,ID=exon:ENST00000456328.2:2;Parent=ENST0000045...
4,chr1,HAVANA,exon,13221,14409,.,+,.,ID=exon:ENST00000456328.2:3;Parent=ENST0000045...


タンパク質をコードする遺伝子（`gene_type="protein_coding"`）については、GFFファイル中に3'UTRの情報(`type="three_prime_RNA"`)が含まれており、その情報を抽出すればOK。一方、lncRNA（`gene_type=lncRNA`）については、GFFファイル中に3'UTRの情報が含まれていないため、代わりに遺伝子全長の情報（`type="transcript"`）を抽出する。GFFファイルには1つの遺伝子あたり複数の転写産物が登録されているが、その中で最も代表的なもの (`Ensembl_canonical`) がEnsemblのアルゴリズムによって選出されている。この**canonical**な転写産物のみを抽出する。
### Ensembl Canonical transcript
The Ensembl Canonical transcript is a single, representative transcript identified at every locus. For accurate analysis, we recommend that more than one transcripts at a locus may need to be considered, however, we designate a single Ensembl Canonical transcript per locus to provide consistency when only one transcript is required e.g. for initial display on the Ensembl (or other) website and for use in genome-wide calculations e.g. the Ensembl gene tree analysis.

For protein-coding genes, we aim to identify the transcript that, on balance, has the highest coverage of conserved exons, highest expression, longest coding sequence and is represented in other key resources, such as NCBI and UniProt. To identify this transcript, we consider, where available, evidence of functional potential (such as evolutionary conservation of a coding region, transcript expression levels), transcript length and evidence from other resources (such as concordance with the APPRIS1 ’principal isoform’ and with the UniProt/Swiss-Prot ‘canonical isoform’).

まず`gene_type`を抽出する関数を用意。

In [17]:
def get_gene_type(df):
    attributes = df.split(";")
    output_type = [s for s in attributes if "gene_type=" in s][0].replace("gene_type=", "")
    return output_type

### (1) タンパク質をコードしている遺伝子のみを抽出
`gene_type` が `protein_coding` かつ `Ensembl_canonical` なものを抽出。

In [18]:
gff["gene_type"] = gff["attributes"].apply(get_gene_type)
gff = gff[gff["gene_type"] == "protein_coding"]

In [19]:
print(f"Number of rows: {len(gff)}")
gff.head()

Number of rows: 2988184


,chr,source,type,start,end,score,strand,phase,attributes,gene_type
57,chr1,HAVANA,gene,65419,71585,.,+,.,ID=ENSG00000186092.7;gene_id=ENSG00000186092.7...,protein_coding
58,chr1,HAVANA,transcript,65419,71585,.,+,.,ID=ENST00000641515.2;Parent=ENSG00000186092.7;...,protein_coding
59,chr1,HAVANA,exon,65419,65433,.,+,.,ID=exon:ENST00000641515.2:1;Parent=ENST0000064...,protein_coding
60,chr1,HAVANA,exon,65520,65573,.,+,.,ID=exon:ENST00000641515.2:2;Parent=ENST0000064...,protein_coding
61,chr1,HAVANA,CDS,65565,65573,.,+,0,ID=CDS:ENST00000641515.2;Parent=ENST0000064151...,protein_coding


In [20]:
gff["type"].unique()

array(['gene', 'transcript', 'exon', 'CDS', 'start_codon', 'stop_codon',
       'five_prime_UTR', 'three_prime_UTR',
       'stop_codon_redefined_as_selenocysteine'], dtype=object)

In [21]:
gff = gff[gff["attributes"].str.contains("Ensembl_canonical")]

print(f"Number of rows: {len(gff)}")

Number of rows: 518563


Coding sequence (type=`CDS`) の情報のみを抽出する。Sourceは `HAVANA` を利用。

In [22]:
gff = gff[(gff["type"] == "CDS") & (gff["source"] == "HAVANA")]
print(f"Number of rows: {len(gff)}")
gff.head()

Number of rows: 197279


,chr,source,type,start,end,score,strand,phase,attributes,gene_type
61,chr1,HAVANA,CDS,65565,65573,.,+,0,ID=CDS:ENST00000641515.2;Parent=ENST0000064151...,protein_coding
64,chr1,HAVANA,CDS,69037,70008,.,+,0,ID=CDS:ENST00000641515.2;Parent=ENST0000064151...,protein_coding
322,chr1,HAVANA,CDS,450740,451678,.,-,0,ID=CDS:ENST00000426406.4;Parent=ENST0000042640...,protein_coding
448,chr1,HAVANA,CDS,685716,686654,.,-,0,ID=CDS:ENST00000332831.5;Parent=ENST0000033283...,protein_coding
978,chr1,HAVANA,CDS,924432,924948,.,+,0,ID=CDS:ENST00000616016.5;Parent=ENST0000061601...,protein_coding


BEDファイルに記載しておく情報として `gene_name` と `gene_id` の2つの情報があればいい。これらの情報を抽出しする関数を作成。

In [23]:
def get_attributes(df):
    attributes = df.split(";")
    gene_name = [s for s in attributes if "gene_name=" in s][0].replace("gene_name=", "")
    gene_id = [s for s in attributes if "gene_id=" in s][0].replace("gene_id=", "")
    return gene_name + "|" + gene_id

In [24]:
gff["gene"] = gff["attributes"].apply(get_attributes)

In [25]:
rows = len(gff)
unique_rows = len(gff["gene"].unique())

print(f"Number of rows: {rows}")
print(f"Unique rows: {unique_rows}")

Number of rows: 197279
Unique rows: 19974


ユニークな遺伝子名が19,974個なのに行数が多い。CDS がエクソンとして分割されているため。CDS　の長さを計算後、それらを足し合わせた数字を ORF の長さとする。

まずBEDファイルに必要な情報 (`chr`, `start`, `end`, `gene`, `score`, `strand`) を抽出

In [59]:
gff = gff[["chr", "start", "end", "gene"]].drop_duplicates().reset_index(drop=True)
gff["length"] = abs(gff["start"] - gff["end"])
gff["ID"] = gff["gene"] + "|" + gff["chr"]
gff

,chr,start,end,gene,length,ID
0,chr1,65565,65573,OR4F5|ENSG00000186092.7,8,OR4F5|ENSG00000186092.7|chr1
1,chr1,69037,70008,OR4F5|ENSG00000186092.7,971,OR4F5|ENSG00000186092.7|chr1
2,chr1,450740,451678,OR4F29|ENSG00000284733.2,938,OR4F29|ENSG00000284733.2|chr1
3,chr1,685716,686654,OR4F16|ENSG00000284662.2,938,OR4F16|ENSG00000284662.2|chr1
4,chr1,924432,924948,SAMD11|ENSG00000187634.13,516,SAMD11|ENSG00000187634.13|chr1
...,...,...,...,...,...,...
197274,chrY,57209532,57209733,WASH6P|ENSG00000182484.15,201,WASH6P|ENSG00000182484.15|chrY
197275,chrY,57209822,57209980,WASH6P|ENSG00000182484.15,158,WASH6P|ENSG00000182484.15|chrY
197276,chrY,57210640,57210792,WASH6P|ENSG00000182484.15,152,WASH6P|ENSG00000182484.15|chrY
197277,chrY,57211552,57211620,WASH6P|ENSG00000182484.15,68,WASH6P|ENSG00000182484.15|chrY


同じ染色体で重複のある遺伝子について 3' UTR の開始位置の最小値と終了位置の最大値を取得しまとめる。

In [64]:
gene_ID = np.array(gff["ID"].unique())
output = pd.DataFrame(columns=["ID", "length"])

for i in gene_ID:
    tmp_df = gff[gff["ID"] == i]
    tmp_ORF = tmp_df["length"].sum() + len(tmp_df)
    add_row = pd.DataFrame([i, tmp_ORF], index=["ID", "length"]).T
    output = pd.concat([output, add_row])

output

,ID,length
0,OR4F5|ENSG00000186092.7|chr1,981
0,OR4F29|ENSG00000284733.2|chr1,939
0,OR4F16|ENSG00000284662.2|chr1,939
0,SAMD11|ENSG00000187634.13|chr1,2535
0,NOC2L|ENSG00000188976.11|chr1,2250
...,...,...
0,BPY2C|ENSG00000185894.8|chrY,321
0,CDY1|ENSG00000172288.8|chrY,1665
0,VAMP7|ENSG00000124333.16|chrY,663
0,IL9R|ENSG00000124334.18|chrY,1566


BEDファイルを保存。

In [67]:
output.to_csv("gencode.v41.orf.length.tsv", sep="\t", index=False, header=True)

### Q. 3'UTRだけで本当にすべてのリードをカバーしきれているのか？
マッピング後のBAMファイルをIGVで見ると、3'UTRだけでなくラストエクソン（またはさらにその1つ前のエクソン）にまでまたがってリードがマッピングされている遺伝子がいくつも見つかる。そういう遺伝子では3'UTRに貼り付いたリードのみをカウントすると、リード数が少なめに見積もられる。これが積み重なると`DESeq2`で発現変動解析を行う際に size factor に影響が出るのでは？